#### Задание 1. Обучить любую модель классификации на датасете IRIS до применения самописного PCA (2 компоненты) и после него. Сравнить качество классификации по отложенной выборке.

In [231]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score


import warnings
warnings.filterwarnings('ignore')

In [232]:
from sklearn import datasets 

In [233]:
iris = datasets.load_iris()

In [234]:
X = iris.data

In [235]:
y = iris.target

In [236]:
def standard_scale(x):
    new_x = (x - x.mean(axis=0)) / x.std(axis=0)
    return new_x

In [237]:
X = standard_scale(X)

In [238]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1, stratify=y)

In [239]:
def evaluate_model(x_train, y_train, x_test, y_test):
    #model = LogisticRegression(random_state=1)
    model = RandomForestClassifier(max_depth = 3, random_state = 1)
    model.fit(x_train, y_train)
    
    train_pred = model.predict(x_train)
    test_pred = model.predict(x_test)
    
    r2_train = r2_score(y_train, train_pred)
    r2_test = r2_score(y_test, test_pred)
    
    print(f"R2 score train: {r2_train:.2f}, \nR2 score test: {r2_test:.2f}")

In [240]:
evaluate_model(X_train, y_train, X_test, y_test)

R2 score train: 0.94, 
R2 score test: 0.97


На тестовой выборке метрика получилась даже лучше, чем на обучающей выборке

**Проведем понижение размерности на основе метода PCA**

In [241]:
cov_matrix = X.T @ X

eig_values, eig_vectors = np.linalg.eig(cov_matrix)

# сформируем список кортежей (собственное значение, собственный вектор)
eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

# и отсортируем список по убыванию собственных значений
eig_pairs.sort(key=lambda x: x[0], reverse=True)

print('Собственные значения и собственные векторы в порядке убывания:')
for i in eig_pairs:
    print(i)

Собственные значения и собственные векторы в порядке убывания:
(437.77467247979916, array([ 0.52106591, -0.26934744,  0.5804131 ,  0.56485654]))
(137.10457072021052, array([-0.37741762, -0.92329566, -0.02449161, -0.06694199]))
(22.01353133569726, array([-0.71956635,  0.24438178,  0.14212637,  0.63427274]))
(3.107225464292839, array([ 0.26128628, -0.12350962, -0.80144925,  0.52359713]))


Оценка доли дисперсии, которая описывается найденными компонентами 

In [242]:
eig_sum = sum(eig_values)
var_exp = [(i / eig_sum) * 100 for i in sorted(eig_values, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print(f'Доля дисперсии, описываемая каждой из компонент \n{var_exp}')

# а теперь оценим кумулятивную (то есть накапливаемую) дисперсию при учитывании каждой из компонент
print(f'Кумулятивная доля дисперсии по компонентам \n{cum_var_exp}')

Доля дисперсии, описываемая каждой из компонент 
[72.96244541329989, 22.850761786701764, 3.6689218892828777, 0.5178709107154734]
Кумулятивная доля дисперсии по компонентам 
[ 72.96244541  95.8132072   99.48212909 100.        ]


2 первые компоненты описывают 95.8% информации. Этого достаточно, чтобы отбросить последние 2 компоненты

In [243]:
# Сформируем вектор весов из собственных векторов, соответствующих первым двум главным компонентам
W = np.hstack([eig_pairs[i][1].reshape(4,1) for i in range(2)])

print(f'Матрица весов W:\n', W)

Матрица весов W:
 [[ 0.52106591 -0.37741762]
 [-0.26934744 -0.92329566]
 [ 0.5804131  -0.02449161]
 [ 0.56485654 -0.06694199]]


In [244]:
# Сформируем новую матрицу "объекты-признаки"
Z = X.dot(W)

In [245]:
Z[:5]

array([[-2.26470281, -0.4800266 ],
       [-2.08096115,  0.67413356],
       [-2.36422905,  0.34190802],
       [-2.29938422,  0.59739451],
       [-2.38984217, -0.64683538]])

Рассчитаем метрику качества модели, обученной на новом наборе данных (2 главные компоненты)

In [246]:
Z_train, Z_test, y_train, y_test = train_test_split(Z, y, test_size = 0.3, random_state = 1, stratify=y)

In [247]:
evaluate_model(Z_train, y_train, Z_test, y_test)

R2 score train: 0.94, 
R2 score test: 0.77


Метрика получилась ниже, чем до применения метода главных компонент. Видимо убранные признаки были значимыми. Попробуем убрать 1 признак (последний)

In [248]:
W2 = np.hstack([eig_pairs[i][1].reshape(4,1) for i in range(3)])

In [249]:
Z2 = X.dot(W2)

In [250]:
Z2_train, Z2_test, y_train, y_test = train_test_split(Z2, y, test_size = 0.3, random_state = 1, stratify=y)

In [251]:
evaluate_model(Z2_train, y_train, Z2_test, y_test)

R2 score train: 0.94, 
R2 score test: 0.83


Метрика на валидацинной выборке все равно хуже, чем если бы мы использовали все признаки. В данном случае получается, что все признаки оказались важны.

#### Задание 2. *Написать свою реализацию метода главных компонент с помощью сингулярного разложения с использованием функции [numpy.linalg.svd()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.svd.html)

Для реализации понижения размерности методом главных компонент с помощью SVD нужно:
- найти сингулярное разложение $X^TX$;
- сформировать из столбцов матрицы $V$, соответствующих $d$ наибольшим сингулярным числам, матрицу весов $W$;
- получить новую матрицу "объекты-признаки", умножив исходную матрицу $X$ на матрицу весов $W$:

$$Z=XW.$$

In [252]:
cov_matrix = X.T@X

#Найдем сингулярное разложение
U, D, V = numpy.linalg.svd(cov_matrix)
# в данном случае 
#V - это транспонированная матрица V.T
#D - диаг элементы матрицы D

In [253]:
D

array([437.77467248, 137.10457072,  22.01353134,   3.10722546])

In [262]:
#Возьмем первые два сингулярных числа,соответственно нам нужны первые 2 столбца матрицы V
# Не забудем что в результате разложение матрица V транспонирована, значит ее надо транспонировать обратно
W = V.T[:, :2]
W

array([[-0.52106591, -0.37741762],
       [ 0.26934744, -0.92329566],
       [-0.5804131 , -0.02449161],
       [-0.56485654, -0.06694199]])

In [265]:
Z = X@W

In [267]:
Z[:5]

array([[ 2.26470281, -0.4800266 ],
       [ 2.08096115,  0.67413356],
       [ 2.36422905,  0.34190802],
       [ 2.29938422,  0.59739451],
       [ 2.38984217, -0.64683538]])

Полученный результат аналогичен разложению матрицы по собственным векторам и собственным значениям (задание 1)